# Résolution problème Thurber

Source: [Thurber.dat](https://www.itl.nist.gov/div898/strd/nls/data/LINKS/DATA/Thurber.dat)

$n=7,\ m=37$

In [ ]:
# Modules and ENLSIP imports

using ForwardDiff, JuMP, Ipopt, BenchmarkTools

include("../src/enlsip_functions_castor2.jl")

In [ ]:
n = 7
m = 37
nb_eq = 0
nb_constraints = 2*n;

In [ ]:
data = [80.574      -3.067
      84.248      -2.981
      87.264      -2.921
      87.195      -2.912
      89.076      -2.840
      89.608      -2.797
      89.868      -2.702
      90.101      -2.699
      92.405      -2.633
      95.854      -2.481
     100.696      -2.363
     101.060      -2.322
     401.672      -1.501
     390.724      -1.460
     567.534      -1.274
     635.316      -1.212
     733.054      -1.100
     759.087      -1.046
     894.206      -0.915
     990.785      -0.714
    1090.109      -0.566
    1080.914      -0.545
    1122.643      -0.400
    1178.351      -0.309
    1260.531      -0.109
    1273.514      -0.103
    1288.339       0.010
    1327.543       0.119
    1353.863       0.377
    1414.509       0.790
    1425.208       0.963
    1421.384       1.006
    1442.962       1.115
    1464.350       1.572
    1468.705       1.841
    1447.894       2.047
    1457.628       2.200]

y = data[:,1]
t = data[:,2];

In [ ]:
# Résidus

function r_i(x::Vector,t::Float64,y::Float64)
    return y - (x[1] + x[2]*t + x[3]*t^2 + x[4]*t^3) / (1 + x[5]*t + x[6]*t^2 + x[7]*t^3)
end

function r(x::Vector)
    return [r_i(x,t[i],y[i]) for i=1:m]
end

resThurber = ResidualsEval(0)

function (resThurber::ResidualsEval)(x::Vector{Float64}, rx::Vector{Float64}, J::Matrix{Float64})
    
    if resThurber.ctrl == 1
        rx[:] = r(x)
    elseif resThurber.ctrl == 2
        J[:] = ForwardDiff.jacobian(r,x)
    end
    return
end

In [ ]:
# Contraintes

function c(x::Vector)
    return [x[1]-1000; 1300 - x[1];
        x[2]-1000; 1500 - x[2];
        x[3]-300; 600 - x[3];
        x[4]; 100- x[4];
        x[5]; 1 - x[5];
        x[6]; 1 - x[6];
        x[7]; 1 - x[7]]
end

consThurber = ConstraintsEval(0)

function (consThurber::ConstraintsEval)(x::Vector{Float64}, cx::Vector{Float64}, A::Matrix{Float64})
    
    if consThurber.ctrl == 1
        cx[:] = c(x)
    elseif consThurber.ctrl == 2
        A[:] = ForwardDiff.jacobian(c,x)
    end
    return
end   

In [ ]:
# x0 = [1300.0, 1500.0, 500.0, 75.0, 1.0, 0.4, 0.05]# 
x0 = [1000.0, 1000.0, 400.0, 40., 0.7, 0.3, 0.03] # 

e = eps(Float64)
se = sqrt(e)
enlsipThurber = enlsip(x0,resThurber,consThurber,n,m,nb_eq,nb_constraints)
@printf "f(x_jul) = %.10e\n\n" enlsipThurber.obj_value

In [ ]:
x0 = [1300.0, 1500.0, 500.0, 75.0, 1.0, 0.4, 0.05]# 
# x0 = [1000.0, 1000.0, 400.0, 40., 0.7, 0.3, 0.03] # 

e = eps(Float64)
se = sqrt(e)
enlsipThurber = enlsip(x0,resThurber,consThurber,n,m,nb_eq,nb_constraints)
# @printf "f(x_jul) = %.10e\n\n" enlsipThurber.obj_value

Model:         Rational Class (cubic/cubic)
               7 Parameters (b1 to b7)

               y = (b1 + b2*x + b3*x**2 + b4*x**3) / 
                   (1 + b5*x + b6*x**2 + b7*x**3)  +  e


          Starting Values                  Certified Values

        Start 1     Start 2           Parameter     Standard Deviation
        
  b1 =   1000        1300          1.2881396800E+03  4.6647963344E+00
  
  b2 =   1000        1500          1.4910792535E+03  3.9571156086E+01
  
  b3 =    400         500          5.8323836877E+02  2.8698696102E+01
  
  b4 =     40          75          7.5416644291E+01  5.5675370270E+00
  
  b5 =      0.7         1          9.6629502864E-01  3.1333340687E-02
  
  b6 =      0.3         0.4        3.9797285797E-01  1.4984928198E-02
  
  b7 =      0.03        0.05       4.9727297349E-02  6.5842344623E-03

Residual Sum of Squares:                    5.6427082397E+03
Residual Standard Deviation:                1.3714600784E+01
Degrees of Freedom:                                30
Number of Observations:                            37



## Visualisation

In [ ]:
str_to_array = (str::String, T::DataType=Float64) -> parse.(T, split(chop(str; head=1, tail=1), ','))

df = DataFrame(CSV.File("iterates.csv", delim=";"))

In [ ]:
f = (x::Vector, t::Number) -> (x[1] + x[2]*t + x[3]*t^2 + x[4]*t^3) / (1 + x[5]*t + x[6]*t^2 + x[7]*t^3)

entry = range(-4,4,1000)
anim = @animate for i ∈ df[!,:iter]
    scatter(t,y,markershape=:cross, xlabel="t", ylabel="y", legend=:none , ylim = (0,1500))
    x = str_to_array(df[i,:x])
    plot!(entry, (t -> f(x,t)).(entry),title="Itération $i")
end
gif(anim, fps = 2)